In [6]:
import pandas as pd
import numpy as np
#import plotly.express as px
from datetime import datetime
import warnings
import json
warnings.filterwarnings("ignore")

In [3]:
sample_tabdata = pd.read_excel('Sample - Superstore (1).xls')
sample_tabdata = sample_tabdata.iloc[:,1:]
newcolmname = sample_tabdata.columns.str.replace(' ', '')
sample_tabdata.columns = list(newcolmname)

In [10]:
#tab_json=sample_tabdata.iloc[:4000,:].to_json(orient ='records')
tab_dict=sample_tabdata.iloc[:4000,:].to_dict()


In [54]:
sample_tabdata.iloc[:4,:].to_dict()

{'OrderID': {0: 'CA-2020-152156',
  1: 'CA-2020-152156',
  2: 'CA-2020-138688',
  3: 'US-2019-108966'},
 'OrderDate': {0: Timestamp('2020-11-08 00:00:00'),
  1: Timestamp('2020-11-08 00:00:00'),
  2: Timestamp('2020-06-12 00:00:00'),
  3: Timestamp('2019-10-11 00:00:00')},
 'ShipDate': {0: Timestamp('2020-11-11 00:00:00'),
  1: Timestamp('2020-11-11 00:00:00'),
  2: Timestamp('2020-06-16 00:00:00'),
  3: Timestamp('2019-10-18 00:00:00')},
 'ShipMode': {0: 'Second Class',
  1: 'Second Class',
  2: 'Second Class',
  3: 'Standard Class'},
 'CustomerID': {0: 'CG-12520', 1: 'CG-12520', 2: 'DV-13045', 3: 'SO-20335'},
 'CustomerName': {0: 'Claire Gute',
  1: 'Claire Gute',
  2: 'Darrin Van Huff',
  3: "Sean O'Donnell"},
 'Segment': {0: 'Consumer', 1: 'Consumer', 2: 'Corporate', 3: 'Consumer'},
 'Country/Region': {0: 'United States',
  1: 'United States',
  2: 'United States',
  3: 'United States'},
 'City': {0: 'Henderson',
  1: 'Henderson',
  2: 'Los Angeles',
  3: 'Fort Lauderdale'},
 'Stat

# Configurations

In [13]:
# Data receviving

_input = {'tabdata':tab_dict,
 'tabparam':{'target_col':'Profit',
            'impcol':['Sales','Quantity'],
            'grp_cols':['Category','Region','State'],
             'recency':['date_col',"no_of_point"],
             'corr_colm_check': None
            },
 'tsdata':None,
 'tsparam':{'target_col_name':None,
            'impcol':None,
            'grp_col':None}  
  }


tab_knowledge_base = ["Max and Min of {0} is {1}. The {0} is correlated with {2} and could be potential reason for max and min.",
                      " {0} is max at {4} {3}. The {0} is correlated with {2} and could be potential reason."]


# Data

In [14]:
class Data:
    
    def __init__(self,_input=None):
        self.input = _input
        self.tabdf = None
        self.ts_df = None
        self.tabparam = None
        self.tspram = None
        
    def parseinput(self):
        if self.input['tabdata'] != None:
            #self.tabdf = pd.read_json(self.input['tabdata'])
            self.tabdf = pd.DataFrame(self.input['tabdata'])
            self.tabparam = self.input['tabparam']

        if self.input['tsdata'] != None:    
            self.ts_df = self.input['tsdata']
            self.tsparam = self.input['tsparam']
                

In [44]:
# Tab Insights

In [15]:
class Nlgtabinsight:
    def __init__ (self, df, tab_param ):
        self.df = df
        self.imp_cols = tab_param['impcol']
        self.target_col = tab_param['target_col']
        self.nm =  tab_param['corr_colm_check']
        self.corr_val = None
        self.max = None
        self.xaxismax_point = None
        self.xaxismax_name = None
        self.min = None
        self.corr_val = None
        self.corr_col_name = None
        
    
#     def read_file(self):
#         df = pd.read_excel('downloads/Sample - Superstore (1).xls')
#         return df
    
    def get_corr(self):
        corr_df = self.df[self.cols]
#         corr_df = df[self.cols + list(target_col)]* We will not need to pass the target col in the col list 
        corr_coeff = pd.DataFrame(corr_df.corr())
        corr_coeff_val = pd.Series(abs(corr_coeff[self.target_col]))
        corr_coeff_tgt = corr_coeff[corr_coeff_val.between(0.3,0.99)]
        corr_val = corr_coeff_tgt[self.target_col].values
        nm = corr_coeff_tgt.columns[corr_coeff_val.between(0.3,0.99)]
#         print(type(corr_coeff_tgt))
        self.nm = nm[0]
        self.corr_val = corr_val[0]
#         return corr_val[0], nm[0]

    def print_val(self):
        if self.nm != None:
            print(f'Corr between {self.nm} and {self.target_col} is {self.corr_val}')
            
    def get_insight(self):
        global tab_knowledge_base
        
        self.tab_insight = tab_knowledge_base[0].format(self.target_col,(self.max,self.min), self.corr_col_name, self.xaxismax_point)
        
        
            
            

# TS insights

In [16]:
if __name__ == '__main__':

    data=Data(_input)
    data.parseinput()
    
    #
    tabinsght = Nlgtabinsight(data.tabdf,data.tab_param)
    tabinsght.print_val()
    

In [17]:
data.tabdf.head()

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country/Region,City,State,PostalCode,Region,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit
0,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2020-138688,2020-06-12,2020-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
